# Import dependencies

In [1]:
import os
import sys

sys.path.insert(0, os.path.dirname(os.getcwd())) 

In [2]:
import time
import gc
import json

import numpy as np
import pandas as pd

from transformers import BertTokenizer, BertModel
from transformers import logging

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

from InputDataset import InputDataset

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence

import matplotlib.pyplot as plt

from torch import cuda

from absa_models.ABSA_BERT_Dropout_CNN_BiLSTM_Linear import ABSA_BERT_Dropout_CNN_BiLSTM_Linear

In [3]:
device = 'cuda' if cuda.is_available() else 'cpu'
logging.set_verbosity_error() 

In [4]:
print(torch.cuda.get_device_name(0))
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory // 1024 ** 3} GB")

NVIDIA GeForce RTX 2060 SUPER
Memory: 8 GB


In [5]:
def clear_memory():
    torch.cuda.empty_cache()

    with torch.no_grad():
        torch.cuda.empty_cache()

    gc.collect()

# Load Data

In [6]:
DATASET = 'ABSA_SemEval16_Restaurants_train.json'

In [7]:
df = pd.json_normalize(json.load(open(DATASET)))

In [8]:
df.head()

,text,tokens,absa_tags
0,Judging from previous posts this used to be a ...,"[Judging, from, previous, posts, this, used, t...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
1,"We, there were four of us, arrived at noon - t...","[We, ,, there, were, four, of, us, ,, arrived,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"They never brought us complimentary noodles, i...","[They, never, brought, us, complimentary, nood...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,The food was lousy - too sweet or too salty an...,"[The, food, was, lousy, -, too, sweet, or, too...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
4,The food was lousy - too sweet or too salty an...,"[The, food, was, lousy, -, too, sweet, or, too...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"


# Train & Validate

In [9]:
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4

EPOCHS = 2

LEARNING_RATE = 1e-5

TRAIN_SPLIT = 0.8

NO_RUNS = 10

In [10]:
SEQ_LEN = 512

In [11]:
BERT_FINE_TUNED_PATH = '../../../results/ABSA/SemEval16 - Task 5 - Restaurants/models/bert_fine_tuned_512.pth'

In [12]:
MODEL_OUTPUT = '../../../results/ABSA/SemEval16 - Task 5 - Restaurants/models/bert_fine_tuned_dropout_cnn_bilstm_linear_512.pth'
STATS_OUTPUT = '../../../results/ABSA/SemEval16 - Task 5 - Restaurants/stats/bert_fine_tuned_dropout_cnn_bilstm_linear_512.csv'

In [13]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [14]:
best_accuracy = 0.0

In [15]:
def create_mini_batch(samples):
    ids_tensors = [s[1] for s in samples]
    ids_tensors[0] = torch.nn.ConstantPad1d((0, SEQ_LEN - len(ids_tensors[0])), 0)(ids_tensors[0])
    ids_tensors = pad_sequence(ids_tensors, batch_first=True).to(device)

    tags_tensors = [s[2] for s in samples]
    tags_tensors[0] = torch.nn.ConstantPad1d((0, SEQ_LEN - len(tags_tensors[0])), 0)(tags_tensors[0])
    tags_tensors = pad_sequence(tags_tensors, batch_first=True).to(device)
    
    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long).to(device)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1).to(device)
    
    return ids_tensors, tags_tensors, masks_tensors

In [16]:
def train(epoch, model, loss_fn, optimizer, dataloader):
    model.train()

    dataloader_len = len(dataloader)

    losses = []

    for _,data in enumerate(dataloader, 0):
        optimizer.zero_grad()

        ids_tensors, tags_tensors, masks_tensors = data

        outputs = model(ids_tensors, masks_tensors)

        loss = loss_fn(outputs.view(-1, 4), tags_tensors.view(-1))

        losses.append(loss.item())
        
        if _ % (dataloader_len // 10) == 0:
            print(f"Epoch: {epoch}/{EPOCHS}, Batch: {_}/{dataloader_len}, Loss: {loss.item()}")
        
        loss.backward()
        
        optimizer.step()
    
    return losses

In [17]:
def validation(model, dataloader):
    model.eval()
    
    fin_targets=[]
    fin_outputs=[]

    with torch.no_grad():
        for _, data in enumerate(dataloader, 0):
            ids_tensors, tags_tensors, masks_tensors = data
            ids_tensors = ids_tensors.to(device)
            tags_tensors = tags_tensors.to(device)
            masks_tensors = masks_tensors.to(device)

            outputs = model(ids_tensors, masks_tensors)
            
            _, predictions = torch.max(outputs, dim=2)

            fin_outputs += list([int(p) for pred in predictions for p in pred])
            fin_targets += list([int(tag) for tags_tensor in tags_tensors for tag in tags_tensor])

    return fin_outputs, fin_targets

In [18]:
results = pd.DataFrame(columns=['accuracy','precision_score_micro','precision_score_macro','recall_score_micro','recall_score_macro','f1_score_micro','f1_score_macro', 'execution_time'])

In [19]:
for i in range(NO_RUNS):
    # clear cache cuda
    torch.cuda.empty_cache()
    with torch.no_grad():
        torch.cuda.empty_cache()
    gc.collect()

    start_time = time.time()

    print(f"Run {i + 1}/{NO_RUNS}")

    train_dataset = df.sample(frac=TRAIN_SPLIT)
    test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    training_set = InputDataset(train_dataset, tokenizer)
    testing_set = InputDataset(test_dataset, tokenizer)

    train_dataloader = DataLoader(
        training_set,
        sampler = RandomSampler(train_dataset),
        batch_size = TRAIN_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    validation_dataloader = DataLoader(
        testing_set,
        sampler = SequentialSampler(testing_set),
        batch_size = VALID_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    model = ABSA_BERT_Dropout_CNN_BiLSTM_Linear(torch.load(BERT_FINE_TUNED_PATH), bert_seq_len=SEQ_LEN, dropout=0.3, bilstm_in_features=256, conv_out_channels=SEQ_LEN, conv_kernel_size=3, no_out_labels=4, device=device).to(device)

    optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)
    loss_fn = torch.nn.CrossEntropyLoss()

    train_losses = []

    for epoch in range(EPOCHS):
        losses = train(epoch, model, loss_fn, optimizer, train_dataloader)

        train_losses += losses
    
    plt.title(f'Train Loss for run {i + 1}/{NO_RUNS}')
    plt.plot(train_losses)
    plt.savefig(f'../../../results/ABSA/SemEval16 - Task 5 - Restaurants/plots/bert_ft_do_cnn_bilstm_lin/train_loss_run_{i + 1}.png')

    plt.clf()

    outputs, targets = validation(model, validation_dataloader)
    
    accuracy = accuracy_score(targets, outputs)
    precision_score_micro = precision_score(targets, outputs, average='micro')
    precision_score_macro = precision_score(targets, outputs, average='macro')
    recall_score_micro = recall_score(targets, outputs, average='micro')
    recall_score_macro = recall_score(targets, outputs, average='macro')
    f1_score_micro = f1_score(targets, outputs, average='micro')
    f1_score_macro = f1_score(targets, outputs, average='macro')

    execution_time = time.time() - start_time

    results.loc[i] = [accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro, execution_time]

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model, MODEL_OUTPUT)

    del train_dataset
    del test_dataset
    del training_set
    del testing_set
    del model
    del loss_fn
    del optimizer
    del outputs
    del targets

Run 1/10
Epoch: 0/2, Batch: 0/501, Loss: 1.3877288103103638
Epoch: 0/2, Batch: 50/501, Loss: 0.7404388785362244
Epoch: 0/2, Batch: 100/501, Loss: 0.24825474619865417
Epoch: 0/2, Batch: 150/501, Loss: 0.12318629026412964
Epoch: 0/2, Batch: 200/501, Loss: 0.10841437429189682
Epoch: 0/2, Batch: 250/501, Loss: 0.05644072964787483
Epoch: 0/2, Batch: 300/501, Loss: 0.042525604367256165
Epoch: 0/2, Batch: 350/501, Loss: 0.03805466368794441
Epoch: 0/2, Batch: 400/501, Loss: 0.037024714052677155
Epoch: 0/2, Batch: 450/501, Loss: 0.08306014537811279
Epoch: 0/2, Batch: 500/501, Loss: 0.02878505550324917
Epoch: 1/2, Batch: 0/501, Loss: 0.040473099797964096
Epoch: 1/2, Batch: 50/501, Loss: 0.039441999047994614
Epoch: 1/2, Batch: 100/501, Loss: 0.03534882515668869
Epoch: 1/2, Batch: 150/501, Loss: 0.028074961155653
Epoch: 1/2, Batch: 200/501, Loss: 0.020895550027489662
Epoch: 1/2, Batch: 250/501, Loss: 0.0238555409014225
Epoch: 1/2, Batch: 300/501, Loss: 0.034720923751592636
Epoch: 1/2, Batch: 350/5

d:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 2/10
Epoch: 0/2, Batch: 0/501, Loss: 1.4066082239151
Epoch: 0/2, Batch: 50/501, Loss: 0.8227415084838867
Epoch: 0/2, Batch: 100/501, Loss: 0.2166071981191635
Epoch: 0/2, Batch: 150/501, Loss: 0.0825473889708519
Epoch: 0/2, Batch: 200/501, Loss: 0.05573026463389397
Epoch: 0/2, Batch: 250/501, Loss: 0.060984622687101364
Epoch: 0/2, Batch: 300/501, Loss: 0.042213473469018936
Epoch: 0/2, Batch: 350/501, Loss: 0.03808578476309776
Epoch: 0/2, Batch: 400/501, Loss: 0.02432713471353054
Epoch: 0/2, Batch: 450/501, Loss: 0.018148360773921013
Epoch: 0/2, Batch: 500/501, Loss: 0.03893457353115082
Epoch: 1/2, Batch: 0/501, Loss: 0.021448533982038498
Epoch: 1/2, Batch: 50/501, Loss: 0.028476005420088768
Epoch: 1/2, Batch: 100/501, Loss: 0.029402997344732285
Epoch: 1/2, Batch: 150/501, Loss: 0.024275710806250572
Epoch: 1/2, Batch: 200/501, Loss: 0.012051149271428585
Epoch: 1/2, Batch: 250/501, Loss: 0.02590612880885601
Epoch: 1/2, Batch: 300/501, Loss: 0.04330853372812271
Epoch: 1/2, Batch: 350/5

d:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 3/10
Epoch: 0/2, Batch: 0/501, Loss: 1.307316780090332
Epoch: 0/2, Batch: 50/501, Loss: 0.5571075081825256
Epoch: 0/2, Batch: 100/501, Loss: 0.15014371275901794
Epoch: 0/2, Batch: 150/501, Loss: 0.0764702633023262
Epoch: 0/2, Batch: 200/501, Loss: 0.048709433525800705
Epoch: 0/2, Batch: 250/501, Loss: 0.035376254469156265
Epoch: 0/2, Batch: 300/501, Loss: 0.03400760143995285
Epoch: 0/2, Batch: 350/501, Loss: 0.03154882416129112
Epoch: 0/2, Batch: 400/501, Loss: 0.02225831337273121
Epoch: 0/2, Batch: 450/501, Loss: 0.03260273113846779
Epoch: 0/2, Batch: 500/501, Loss: 0.03403220325708389
Epoch: 1/2, Batch: 0/501, Loss: 0.009833565913140774
Epoch: 1/2, Batch: 50/501, Loss: 0.058606766164302826
Epoch: 1/2, Batch: 100/501, Loss: 0.021009957417845726
Epoch: 1/2, Batch: 150/501, Loss: 0.017653094604611397
Epoch: 1/2, Batch: 200/501, Loss: 0.02453366108238697
Epoch: 1/2, Batch: 250/501, Loss: 0.01893358863890171
Epoch: 1/2, Batch: 300/501, Loss: 0.013695674017071724
Epoch: 1/2, Batch: 350

...

Run 8/10
Epoch: 0/2, Batch: 0/501, Loss: 1.371658205986023
Epoch: 0/2, Batch: 50/501, Loss: 0.855475664138794
Epoch: 0/2, Batch: 100/501, Loss: 0.31971001625061035
Epoch: 0/2, Batch: 150/501, Loss: 0.15657441318035126
Epoch: 0/2, Batch: 200/501, Loss: 0.10560111701488495
Epoch: 0/2, Batch: 250/501, Loss: 0.09046781808137894
Epoch: 0/2, Batch: 300/501, Loss: 0.06466003507375717
Epoch: 0/2, Batch: 350/501, Loss: 0.053085412830114365
Epoch: 0/2, Batch: 400/501, Loss: 0.05087431147694588
Epoch: 0/2, Batch: 450/501, Loss: 0.04656688868999481
Epoch: 0/2, Batch: 500/501, Loss: 0.04053430259227753
Epoch: 1/2, Batch: 0/501, Loss: 0.06078040972352028
Epoch: 1/2, Batch: 50/501, Loss: 0.05132310092449188
Epoch: 1/2, Batch: 100/501, Loss: 0.03103068843483925
Epoch: 1/2, Batch: 150/501, Loss: 0.045772846788167953
Epoch: 1/2, Batch: 200/501, Loss: 0.04882192984223366
Epoch: 1/2, Batch: 250/501, Loss: 0.04208146035671234
Epoch: 1/2, Batch: 300/501, Loss: 0.043097347021102905
Epoch: 1/2, Batch: 350/501

d:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 9/10
Epoch: 0/2, Batch: 0/501, Loss: 1.3428397178649902
Epoch: 0/2, Batch: 50/501, Loss: 0.7533325552940369
Epoch: 0/2, Batch: 100/501, Loss: 0.22322529554367065
Epoch: 0/2, Batch: 150/501, Loss: 0.12167410552501678
Epoch: 0/2, Batch: 200/501, Loss: 0.052280329167842865
Epoch: 0/2, Batch: 250/501, Loss: 0.04178668558597565
Epoch: 0/2, Batch: 300/501, Loss: 0.0591340996325016
Epoch: 0/2, Batch: 350/501, Loss: 0.026799004524946213
Epoch: 0/2, Batch: 400/501, Loss: 0.03164544329047203
Epoch: 0/2, Batch: 450/501, Loss: 0.02397077903151512
Epoch: 0/2, Batch: 500/501, Loss: 0.01517350971698761
Epoch: 1/2, Batch: 0/501, Loss: 0.02982010319828987
Epoch: 1/2, Batch: 50/501, Loss: 0.026152588427066803
Epoch: 1/2, Batch: 100/501, Loss: 0.026748212054371834
Epoch: 1/2, Batch: 150/501, Loss: 0.02760486863553524
Epoch: 1/2, Batch: 200/501, Loss: 0.019757000729441643
Epoch: 1/2, Batch: 250/501, Loss: 0.021311897784471512
Epoch: 1/2, Batch: 300/501, Loss: 0.03877515718340874
Epoch: 1/2, Batch: 350

d:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


<Figure size 432x288 with 0 Axes>

In [20]:
results

,accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro,execution_time
0,0.995164,0.995164,0.248791,0.995164,0.250000,0.995164,0.249394,384.131582
1,0.995117,0.995117,0.454240,0.995117,0.275414,0.995117,0.294657,394.750716
2,0.994734,0.994734,0.457032,0.994734,0.258099,0.994734,0.264943,376.528235
3,0.995047,0.995047,0.548766,0.995047,0.251881,0.995047,0.253108,372.622299
4,0.995023,0.995023,0.248756,0.995023,0.250000,0.995023,0.249376,387.927254
5,0.995223,0.995223,0.428305,0.995223,0.257758,0.995223,0.264284,398.892684
6,0.995078,0.995078,0.248772,0.995078,0.250000,0.995078,0.249385,381.397712
7,0.995117,0.995117,0.248779,0.995117,0.250000,0.995117,0.249388,371.586913
8,0.995477,0.995477,0.498881,0.995477,0.250568,0.995477,0.250577,372.294208
9,0.994836,0.994836,0.248725,0.994836,0.250000,0.994836,0.249361,376.453127


In [21]:
results.to_csv(STATS_OUTPUT)